<a href="https://colab.research.google.com/github/scorepia/Realtime6005_Project/blob/main/DADS6005_realtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DADS6005 - Analyze by Reddit Data

##Reddit API

In [1]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 3.5 MB/s eta 0:00:00


In [2]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Install PySpark and Spark NLP
!pip install -q pyspark==3.1.2 spark-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.5/453.5 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 KB 17.9 MB/s eta 0:00:00


## Import modules and libraries

In [4]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
import praw
import requests

In [5]:
#setting reddit api key
reddit = praw.Reddit(
    client_id='XVbrwUxDwMhsTdhig-xEEQ',
    client_secret='ZyKvx4d4DUd0ixx0aLvygUNhls4Tdw',
    user_agent='DADS6005_Realtime',
)

## Scrapping topics data from subreddit "Cocacola and Pepsi"

In [6]:
#test btc post on subreddit time filter all
BTC_posts = reddit.subreddit('cocacola').hot()

list_title = []
for post in BTC_posts:
    print(post.title)
    list_title.append(post.title)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Found these at my local Mexican restaurant today. I've never seen them before so I bought some haven't tried it yet.
Has anyone seen this in our American stores? I found it in one of the stores in Ukraine.
How wide and tall is a glass Coca Cola cap
Here’s something I created, inspired by my love of Coca Cola
Coca-Cola, 1971 - 'Hilltop' | "I'd like to buy the world a Coke"
Coca Cola 'Have A Coke and a Smile' 1970's TV Commercial
anyone know anything about this?
Cafe de Coral HK 大家樂
I prefer the sugar versions
Anybody Knows what this is worth I found it in my woods ( Swerve Vanana Can)
What the heck is this??
Has anyone made peace with Coke Zero sugar?
who wants this back?
Coca-Cola from plastic bottles tastes awful
Cola Bears
Found a golden cap while drinking coke, does this mean anything?
Unintentionally created a Coca Cola commercial
Can someone help me how old is it?
🧧
Can't wait for customers to ask " aY iS tHiS RrGuLaR SpRiTe Or sPrItE ZeRo?
Got the McD x Coca Cola glass from my br

##Spark NLP with Pretrain model

### Sentiment Analysis base on Twitter 

In [7]:
spark = sparknlp.start()

In [8]:
#define pretrain model 'sentimentdl_use_imdb'
MODEL_NAME='sentimentdl_use_twitter'

In [9]:
#define nlp pipeline 
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


In [10]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

df = spark.createDataFrame(pd.DataFrame({"text":list_title}))
result = pipelineModel.transform(df)

In [11]:
df_sentiment=result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("post title"),
        F.expr("cols['1']").alias("sentiment")).show(20,truncate=False)

+--------------------------------------------------------------------------------------------------------------------+---------+
|post title                                                                                                          |sentiment|
+--------------------------------------------------------------------------------------------------------------------+---------+
|Found these at my local Mexican restaurant today. I've never seen them before so I bought some haven't tried it yet.|positive |
|Has anyone seen this in our American stores? I found it in one of the stores in Ukraine.                            |positive |
|How wide and tall is a glass Coca Cola cap                                                                          |negative |
|Here’s something I created, inspired by my love of Coca Cola                                                        |positive |
|Coca-Cola, 1971 - 'Hilltop' | "I'd like to buy the world a Coke"                                

In [12]:
twitter_DF = result.toPandas()

###Sentiment Analysis (Vivekn)

In [13]:
document = DocumentAssembler() \
.setInputCol("text") \
.setOutputCol("document")

token = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token")

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normal")

vivekn =  ViveknSentimentModel.pretrained() \
.setInputCols(["document", "normal"]) \
.setOutputCol("result_sentiment")

finisher = Finisher() \
.setInputCols(["result_sentiment"]) \
.setOutputCols("final_sentiment")

pipeline = Pipeline().setStages([document, token, normalizer, vivekn, finisher])

df2 = spark.createDataFrame(pd.DataFrame({"text":list_title}))
pipelineModel = pipeline.fit(df2)
result2 = pipelineModel.transform(df2)

result2.select("text","final_sentiment").show(20,truncate=False)

sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[OK!]
+--------------------------------------------------------------------------------------------------------------------+---------------+
|text                                                                                                                |final_sentiment|
+--------------------------------------------------------------------------------------------------------------------+---------------+
|Found these at my local Mexican restaurant today. I've never seen them before so I bought some haven't tried it yet.|[negative]     |
|Has anyone seen this in our American stores? I found it in one of the stores in Ukraine.                            |[negative]     |
|How wide and tall is a glass Coca Cola cap                                                                          |[positive]     |
|Here’s something I created, inspired by my love of Coca Cola                          

### Sentiment Analysis base on Imdb

In [14]:
model_imdb = 'sentimentdl_use_imdb'

In [15]:
#define nlp pipeline 
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=model_imdb, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


In [16]:
empty_df_3 = spark.createDataFrame([['']]).toDF("text")

pipelineModel_3 = nlpPipeline.fit(empty_df_3)

df_3 = spark.createDataFrame(pd.DataFrame({"text":list_title}))
result_3 = pipelineModel.transform(df_3)

In [17]:
pandasDF = result_3.toPandas()

In [18]:
pandasDF.head(20)

,text,final_sentiment
0,Found these at my local Mexican restaurant tod...,[negative]
1,Has anyone seen this in our American stores? I...,[negative]
2,How wide and tall is a glass Coca Cola cap,[positive]
3,"Here’s something I created, inspired by my lov...",[positive]
4,"Coca-Cola, 1971 - 'Hilltop' | ""I'd like to buy...",[positive]
5,Coca Cola 'Have A Coke and a Smile' 1970's TV ...,[negative]
6,anyone know anything about this?,[negative]
7,Cafe de Coral HK 大家樂,[na]
8,I prefer the sugar versions,[positive]
9,Anybody Knows what this is worth I found it in...,[negative]


In [19]:
show_result = pandasDF['final_sentiment'].value_counts()

print(type(show_result))


<class 'pandas.core.series.Series'>


In [20]:
#define result data
neg = show_result[0] 
pos = show_result [1]

print(neg,pos)

50 32


### Sentiment Analysis - Pepsi

In [21]:
#test btc post on subreddit time filter all
pepsi_posts = reddit.subreddit('Pepsi').hot()

pepsi_title = []
for post in pepsi_posts:
    #print(post.title)
    pepsi_title.append(post.title)

print(pepsi_title)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



['Anyone know how old this might be? I believe it is an old Pepsi bottle cap', 'We all know about Mexican Coke, but...is Mexican Pepsi a thing?', 'I yearn for Wild Cherry Pepsi Nitro', 'trying new sierra mist replacement/ new pepsi sprite competitor', 'Pepsi says Zero Sugar formula was changed due to lack of ingredients', 'Pepsi Cd case I found at a resale store 50cents!', 'Is anyone here a union Pepsi sales rep? If so please DM me I have some questions. Thank you', 'Pepsi Cola Soda Shop 2023 Edition', 'Please support my change.org to bring back crystal pepsi!', "Imma keep buying this again and again so they keep it around forever and you aren't gonna do anything about it 😈😈😈😈", 'Contact Form to let Pepsi know we want the original Zero Sugar formula back 👇', 'Mountain Dew pitch black it’s black review of that new new￼￼', 'I found it it’s back', 'The new Pepsi Zero Sugar is great!', 'Is this caffeine figure real? The 2liter bottle has only half as much per 12oz servings.', 'Starry Sucks

In [22]:
#define nlp pipeline 
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=model_imdb, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


In [23]:
empty_df_4 = spark.createDataFrame([['']]).toDF("text")

pipelineModel_4 = nlpPipeline.fit(empty_df_4)

df_4 = spark.createDataFrame(pd.DataFrame({"text":pepsi_title}))
result_4 = pipelineModel.transform(df_4)

In [24]:
result_4.show()

+--------------------+---------------+
|                text|final_sentiment|
+--------------------+---------------+
|Anyone know how o...|     [positive]|
|We all know about...|     [negative]|
|I yearn for Wild ...|     [positive]|
|trying new sierra...|     [negative]|
|Pepsi says Zero S...|     [negative]|
|Pepsi Cd case I f...|     [positive]|
|Is anyone here a ...|     [negative]|
|Pepsi Cola Soda S...|     [positive]|
|Please support my...|     [positive]|
|Imma keep buying ...|     [negative]|
|Contact Form to l...|     [positive]|
|Mountain Dew pitc...|     [negative]|
|I found it it’s back|     [positive]|
|The new Pepsi Zer...|     [positive]|
|Is this caffeine ...|     [negative]|
|        Starry Sucks|     [negative]|
|is starry replaci...|     [positive]|
|Mountain Dew Pitc...|     [negative]|
|         New formula|     [negative]|
|Is PepsiCola crea...|     [positive]|
+--------------------+---------------+
only showing top 20 rows



In [25]:
df_pandas_pepsi = result_4.toPandas()

In [26]:
show_result_pepsi = df_pandas_pepsi['final_sentiment'].value_counts()

print(show_result_pepsi)

[negative]    56
[positive]    36
[na]           8
Name: final_sentiment, dtype: int64


In [27]:
pandasDF

,text,final_sentiment
0,Found these at my local Mexican restaurant tod...,[negative]
1,Has anyone seen this in our American stores? I...,[negative]
2,How wide and tall is a glass Coca Cola cap,[positive]
3,"Here’s something I created, inspired by my lov...",[positive]
4,"Coca-Cola, 1971 - 'Hilltop' | ""I'd like to buy...",[positive]
...,...,...
95,"A Kind of Magic - Danna Paola, Felukah, Tamtam...",[negative]
96,Dreamworld Coke That I Had When I Went To The ...,[positive]
97,anyone know where I can find the 35 can pack f...,[positive]
98,90's me looking like I was plucked from a coca...,[negative]


In [ ]:
sentiment_json = 

In [85]:
list_line = []

dict_coca = pd.DataFrame({
    'name': 'Cocacola',
    'Negative ' : [show_result[0]], 
    'Positive' : [show_result[1]],
    'General' : [show_result[2]]
})


j = dict_coca.loc[0]
print(j)
json_coca = j.to_json()
json_encode = json_coca.encode()


dict_pepsi = pd.DataFrame({
    'name' : 'Pepsi',
    'Negative ' : [show_result_pepsi[0]], 
    'Positive' : [show_result_pepsi[1]],
    'General' : [show_result_pepsi[2]]
})

y = dict_pepsi.loc[0]
#print(y)
json_pepsi = y.to_json()
json_encode2 = json_pepsi.encode()



list_line.append(json_encode)
list_line.append(json_encode2)

name         Cocacola
Negative           50
Positive           32
General            16
Name: 0, dtype: object


## Kafka Producer 

In [28]:
!pip install confluent_kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.9 MB/s eta 0:00:00


In [102]:

from confluent_kafka import Producer
import time
import json
from uuid import uuid4

p = Producer({'bootstrap.servers': 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'})

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {}'.format(msg.value().decode('utf-8')))


for line in list_line:
    p.poll(0)
    sendMsg = line # line.encode().decode('utf-8').strip('\n')
    p.produce('producer3', key=str(uuid4()), value=sendMsg , callback=delivery_report)
    #time.sleep(5)

p.flush()

Message delivered to {"name":"Cocacola","Negative ":50,"Positive":32,"General":16}
Message delivered to {"name":"Pepsi","Negative ":56,"Positive":36,"General":8}


0